In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.signal

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
ls 'gdrive/My Drive/left_fist.npy'

'gdrive/My Drive/left_fist.npy'


In [0]:
left_fist = np.load('gdrive/My Drive/left_fist.npy')
right_fist = np.load('gdrive/My Drive/right_fist.npy')
both_fist = np.load('gdrive/My Drive/both_fist.npy')
both_feet = np.load('gdrive/My Drive/both_feet.npy')

In [0]:
left_fist = np.array(left_fist[:530])
right_fist = np.array(right_fist[:530])
both_fist = np.array(both_fist)
both_feet = np.array(both_feet)
print(left_fist.shape)
print(right_fist.shape)
print(both_fist.shape)
print(both_feet.shape)

(530, 64, 113)
(530, 64, 113)
(530, 64, 113)
(530, 64, 113)


In [0]:
print(abs(np.sum(left_fist[0][0] - right_fist[0][0])))
print(abs(np.sum(left_fist[0][0] - both_fist[0][0])))
print(abs(np.sum(left_fist[0][0] - both_feet[0][0])))
print(abs(np.sum(both_feet[0][0] - both_fist[0][0])))

6.853359500868575e-07
1.6039158473338503e-05
3.0591367277883825e-05
1.4552208804545327e-05


In [0]:
def preprocessing(data):
    mean = np.mean(data, axis = 0)    
    var = np.var(data, axis = 0)
    data = (data - mean)/var
    return data
#data_array = preprocessing(data)

In [0]:
events = np.zeros((2120,4))
ones = np.ones((530,))

# left fist
events[:530,0] = events[:530,0] + ones
# right fist
events[530:1060,1] = events[530:1060,1] + ones
# both fist 
events[1060:1590,2] = events[1060:1590,2] + ones
# both feet
events[1590:,3] = events[1590:,3] + ones

In [0]:
left_fist1 = abs(left_fist.reshape((530, 7232)))
right_fist1 = abs(right_fist.reshape((530, 7232)))
both_fist1 = abs(both_fist.reshape((530, 7232)))
both_feet1 = abs(both_feet.reshape((530, 7232)))

left_fist1 = np.multiply(left_fist1, left_fist1)
right_fist1 = np.multiply(right_fist1, right_fist1)
both_fist1 = np.multiply(both_fist1, both_fist1)
both_feet1 = np.multiply(both_feet1, both_feet1)

left_fist1 = preprocessing(left_fist1)
right_fist1 = preprocessing(right_fist1)
both_fist1 = preprocessing(both_fist1)
both_feet1 = preprocessing(both_feet1)

In [0]:
print(left_fist1.shape)
print(right_fist1.shape)
print(both_fist1.shape)
print(both_feet1.shape)

(530, 7232)
(530, 7232)
(530, 7232)
(530, 7232)


In [0]:
power_data = np.concatenate((left_fist1, right_fist1, both_fist1, both_feet1), axis = 0)
power_data.shape

(2120, 7232)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler # Used for scaling of data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
x_train, x_test, y_train, y_test = train_test_split(power_data, events, test_size=0.3, random_state=42)
y_test.shape

(636, 4)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p=2)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
from sklearn.metrics import confusion_matrix
import tensorflow as tf
def result(x_test, y_test):
  y_pred = classifier.predict(x_test)
  y_classes = y_pred.argmax(axis=-1)
  label =  tf.keras.utils.to_categorical(np.asarray(y_classes)) 
  matrix = confusion_matrix(y_test.argmax(axis=1), label.argmax(axis=1))
  FP = np.sum(matrix.sum(axis=0) - np.diag(matrix))/4  
  FN = np.sum(matrix.sum(axis=1) - np.diag(matrix))/4
  TP = np.sum(np.diag(matrix))/4
  TN = np.sum(matrix.sum() - (FP + FN + TP))/4
  data = []
  # Overall accuracy
  accuracy = (TP+TN)/(TP+FP+FN+TN)
  data.append(accuracy)
  #Error
  error = (FP + FN)/(TP + TN + FN + FP)
  data.append(error)
  # Sensitivity, hit rate, recall, or true positive rate
  sensitivity = TP/(TP+FN)  
  data.append(sensitivity)
  # Specificity or true negative rate
  specificity = TN/(TN+FP)
  data.append(specificity)
  # Precision or positive predictive value
  precision = TP/(TP+FP)
  data.append(precision)
  # Fall out or false positive rate
  false_positive_rate = FP/(FP+TN)
  data.append(false_positive_rate)

  #F-score
  f_score = (2*(precision *sensitivity))/(precision  + sensitivity)
  data.append(f_score)
  # False discovery rate
  FDR = FP/(TP+FP)
  data.append(FDR)
  # Negative predictive value
  NPV = TN/(TN+FN)
  data.append(NPV)
  return data
test_results = result(x_test, y_test)
test_results

[0.6011461318051576,
 0.3988538681948424,
 0.589622641509434,
 0.6120401337792643,
 0.589622641509434,
 0.3879598662207358,
 0.589622641509434,
 0.41037735849056606,
 0.6120401337792643]

In [0]:
train_results = result(x_train, y_train)
train_results

[0.6576606813223695,
 0.34233931867763046,
 0.6563342318059299,
 0.6589769307923772,
 0.6563342318059299,
 0.3410230692076229,
 0.6563342318059299,
 0.3436657681940701,
 0.6589769307923772]